# Momento de Retroalimentación: Módulo 1 Construcción de un modelo estadístico base (Portafolio Implementación) 

## 2.2.1 About descriptive statistics

### 2.2.1.1 Considering the most recent financial quarter of the dataset

In [49]:
# Importar las librerias necesarias

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from symbulate import *
import plotly.express as px

In [50]:
# Importar los archivos
df = pd.read_csv("us2022q2a.csv")
df2 = pd.read_csv("usfirms2022.csv")

In [51]:
# Dividirlo por periodo
df['qdate'] = pd.PeriodIndex(df['q'], freq="Q")

In [52]:
# Establecer qdate y firm como index
df.set_index(['qdate','firm'], inplace=True)

In [53]:
# Extraer sólo las columnas necesarias
df_firm = df2[['Ticker', 'Name', 'Sector NAICS\nlevel 1']]

#### Show how many firms by industry there are in the sample:
Cada empresa hay un total de 90 Ticker

In [54]:
# Merge con otra tabla para ver la relación entre 'firm' y 'Ticker'
df_merge = pd.merge(df, df_firm, left_on='firm', right_on='Ticker', how='left')
df_merge.groupby('Ticker').count()
# df_merge.groupby('Ticker').count().head()

,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,totalliabilities,...,longdebt,stockholderequity,adjprice,originalprice,sharesoutstanding,fiscalmonth,year,cto,Name,Sector NAICS\nlevel 1
Ticker,,,,,,,,,,,,,,,,,,,,,
A,90,87,87,87,87,87,87,87,89,89,...,89,89,90,90,90,89,90,90,90,90
AA,90,24,24,24,24,24,24,24,24,24,...,24,24,23,23,23,24,90,90,90,90
AAIC,90,78,78,78,78,78,78,78,78,78,...,78,78,90,90,77,78,90,90,90,90
AAL,90,90,90,90,90,90,90,90,90,90,...,90,90,35,35,90,90,90,90,90,90
AAME,90,45,45,45,45,45,45,45,45,45,...,45,45,46,46,44,45,90,90,90,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZVIA,90,5,5,5,5,5,5,5,5,5,...,5,5,4,4,4,5,90,90,90,90
ZVO,90,45,45,45,45,45,45,45,45,45,...,45,45,46,46,44,45,90,90,90,90
ZWS,90,41,41,41,41,41,41,41,42,42,...,42,42,42,42,41,42,90,90,90,90


#### For each industry (and for all industries), what can you say about the typical firm size in terms of market value and book value? How much these variables change within each industry? How firm size (in market value) is distributed?

In [60]:
df_merge['Markt_Value'] = 0
df_merge['Book_Value'] = 0
# Markt value de cada empresa por año
i = 0
for i in range(i, len(df_merge)):
    if df_merge['q'][i][5] == '4':
        # print('originalprice: ', df_merge['originalprice'][i])
        # print("Markt_Value: ", df_merge['originalprice'][i]*df_merge['sharesoutstanding'][i])
        df_merge['Markt_Value'][i] = df_merge['originalprice'][i] * df_merge['sharesoutstanding'][i]
        df_merge['Book_Value'][i] = df_merge['totalassets'][i] - df_merge['totalliabilities'][i]
df_merge[['Markt_Value','Book_Value']].head()

C:\Users\Keyuan\AppData\Local\Temp\ipykernel_18160\196650180.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['Markt_Value'][i] = df_merge['originalprice'][i] * df_merge['sharesoutstanding'][i]
C:\Users\Keyuan\AppData\Local\Temp\ipykernel_18160\196650180.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['Book_Value'][i] = df_merge['totalassets'][i] - df_merge['totalliabilities'][i]


,Markt_Value,Book_Value
0,0.000000e+00,0.0
1,0.000000e+00,0.0
2,0.000000e+00,0.0
3,2.498606e+07,5265000.0
4,0.000000e+00,0.0


In [62]:
df_year = df_merge.groupby(['Ticker','year'])[['Markt_Value','Book_Value']].last()
df_year.describe()

,Markt_Value,Book_Value
count,8.276400e+04,8.276400e+04
mean,4.708704e+06,1.618159e+06
std,3.267875e+07,9.136523e+06
min,0.000000e+00,-8.534000e+07
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,1.259100e+06,4.959865e+05
max,2.913284e+09,2.941270e+08
